# Import Libraries and Datasets

In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import tensorflow as tf

from src.constants import IMAGE_SIZE, CHANNELS
from src.preprocessing_functions import create_generator
from src.models import build_cnn_model, train_cnn_model, plot_training_validation_history, evaluate_train_test_performance

In [2]:
# Load paths in a dict for train, test, valid folders with wildfire and nowildfire images 

images_path = '/Users/silvanoquarto/Desktop/PROJECTS/Data_Wildfire_Project'
splits = ['train', 'test', 'valid']
labels = ['wildfire', 'nowildfire']

image_paths = {f"{split}_{label}": [] for split in splits for label in labels}

for split in splits:
    for label in labels:
        path = os.path.join(images_path, split, label)
        if os.path.exists(path):  
            image_files = os.listdir(path)
            full_paths = [os.path.join(path, img) for img in image_files]
            image_paths[f"{split}_{label}"].extend(full_paths)
            print(f"- {split.capitalize()} set for {label} uploaded correctly!!")
        else:
            print(f"Warning: {path} does not exist :(")

- Train set for wildfire uploaded correctly!!
- Train set for nowildfire uploaded correctly!!
- Test set for wildfire uploaded correctly!!
- Test set for nowildfire uploaded correctly!!
- Valid set for wildfire uploaded correctly!!
- Valid set for nowildfire uploaded correctly!!


# IMAGE PREPROCESSING

In [3]:
# Data augumentation and normalization
 
print('Data augumentation and normalization for training...')
train_generator = create_generator(images_path, 'train')

print('Data augumentation and normalization for validation...')
validation_generator = create_generator(images_path, 'valid')

print('Data augumentation and normalization for test...')
test_generator = create_generator(images_path, 'test')

Data augumentation and normalization for training...
Found 30249 images belonging to 2 classes.
Data augumentation and normalization for validation...
Found 6300 images belonging to 2 classes.
Data augumentation and normalization for test...
Found 6299 images belonging to 2 classes.


# Build and Train CNN model from scratch

## Build model

In [4]:
model = build_cnn_model(input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS), num_output=1, learning_rate=0.001, dropout_rate=0.5)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 348, 348, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 174, 174, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 172, 172, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 86, 86, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 84, 84, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 42, 42, 128)       0

## Train model

In [2]:
# Check if GPU is available

print(f"Tensor Flow Version: {tf.__version__}")
print()
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.16.1

GPU is available


In [ ]:
history = train_cnn_model(
    model,
    train_generator,
    validation_generator,
    epochs=20
)

# Model evaluation

In [ ]:
# Training vs Validation performances

plot_training_validation_history(history)

In [ ]:
# Training vs Test performance

evaluate_train_test_performance(model, train_generator, test_generator)

# Save model

In [ ]:
# Save the model

model.save('wildfire_detection_cnn.h5')